# Python-Umgebung

In diesem Abschnitt wird die Umgebung für das Arbeiten im Notebook vorbereitet. 

## Import aller verwendeten Bibliotheken

Zunächst werden alle relevanten Pakete für Python importiert:

In [ ]:
import numpy as np
from scipy.optimize import fsolve, root
from scipy import signal
import matplotlib.pyplot as plt

## Standard-Look für Plots

Dann werden einige Standard-Einstellungen für das Paket *matplotlib* gesetzt. Diese beeinflussen alle Plots die wir erstellen - so können später mit kleinen Änderungen in diesem Bereich alle Plots gleichzeitig angepasst werden. Typische Beispiele sind Schriftgrößen, Linienstärken etc. Eine Übersicht über die Einstellungsmöglichkeiten findet man z.B. [hier](https://matplotlib.org/stable/tutorials/introductory/customizing.html).

In [ ]:
# Dieser Befehl öffnet ein neues Fenster für Plots anstatt sie im Notebook anzuzeigen:
#%matplotlib qt

# Optische Einstellungen
plt.rcParams['figure.figsize'] = [15,6]
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.grid'] = True
plt.rcParams['font.size'] = 14


# Theorie

## Der Bingham-Hooke-Körper

Als rheologisches Modell für *Viskoplastizität* betrachten wir den *Bingham-Hooke-Körper*:
<p><img src="bingham-hooke.png" width="400"/></p>

Wie bei der Viskoelastizität wird wiederum entweder die Spannung $\sigma$ oder die Dehnung $\varepsilon$ vorgeschrieben und die jeweils andere Größe berechnet. Im Folgenden werden, wo nicht explizit andere Werte genannt werden, die folgenden <b>Materialparameter</b> verwendet:

| Element               | Parameter  | Wert | Einheit |
|:---------------------:|:----------:|:----:|:-------:|
| Hooke-Element         | $E$        | 200  | MPa     |
| Newton-Element        | $\eta$     |  50  | MPa s   |
| St.-Venant-Element    | $\sigma_y$ |  10  | MPa     |

Die Spannungen und Dehnungen an den einzelnen Elementen bezeichnen wir wie folgt:

| Element                         | Dehnung               | Spannung            |
|:-------------------------------:|:---------------------:|:-------------------:|
| Hooke-Element ($E$)             | $\varepsilon_e$       | $\sigma_e$          |
| Newton-Element ($\eta$)         | $\varepsilon_v$       | $\sigma_v$          |
| St.-Venant-Element ($\sigma_y$) | $\varepsilon_p$       | $\sigma_p$          |

Aus den Beziehungen für die Reihen-/Parallelschaltung ergeben sich folgende Gleichungen:
$$\begin{aligned}
\varepsilon_v & = \varepsilon_p \\
\varepsilon & = \varepsilon_e + \varepsilon_v \\
\sigma & = \sigma_e \\
\sigma_e & = \sigma_v + \sigma_p
\end{aligned}$$

Da die Dehnungen des Newton-Elements und des St.-Venant-Elements identisch sind, bezeichnen wir diese im Folgenden als $\varepsilon_{vp}$ und wählen diese Größe als interne Variable. Die dazugehörige Spannung ergibt sich als $\sigma_{vp} = \sigma_v + \sigma_p$.

Für die Spannungs-Dehnungs-Beziehung gelten an den drei Elementen die folgenden Gleichungen:
$$\begin{aligned}
\sigma_e & = E \, \varepsilon_e \\
\sigma_v & = \eta \, \dot \varepsilon_{vp} \\
\sigma_p & \left\{ \begin{matrix}
    = +\sigma_y               & \text{für } \dot \varepsilon_{vp} > 0 \\
    \in \left[-\sigma_y, \sigma_y\right] & \text{für } \dot \varepsilon_{vp} = 0 \\
    =-\sigma_y                & \text{für } \dot \varepsilon_{vp} < 0
    \end{matrix} \right.
\end{aligned}$$



Für die Dehnung des Körpers gilt der Zusammenhang
$$\varepsilon = \varepsilon_{vp} + \varepsilon_e \ ,$$
wobei $\varepsilon_{vp} = \varepsilon_{v} = \varepsilon_{p}$ die Dehnungen des Dämpfers bzw. des Reibelements und $\varepsilon_e$ die Dehnung der Feder darstellen. Die Dehnung des Reibelements ist identisch zur Dehnung des Dämpfers. Im Weiteren wird die Dehnung des Dämpfers $\varepsilon_{vp}$ als interne Variable gewählt.

Die Evolutionsleichung kann durch drei Gleichungen ausgedrückt werden:
$$\begin{aligned}
\dot \varepsilon_{vp} & = \lambda \frac{\sigma}{\left|\sigma\right|} \ ,\\
\lambda & = \frac{\max\left\{0,\Phi\right\}}{\eta} \geq 0 \\
\Phi & = \left|\sigma \right| - \sigma_y
\end{aligned}$$


## Numerische Integration

Die Evolutionsgleichung wird mittels der impliziten Euler-Regel integriert:
$$\frac{\varepsilon_{vp,n+1} - \varepsilon_{vp,n}}{\Delta t} = \dot \varepsilon_{vp,n+1}$$

Um zwischen *elastischen* und *viskoplastischen* Zuständen zu unterscheiden, wird eine so genannten *Predictor-Corrector-Methode* verwendet. Zunächst werden, unter der Annahme dass der Zeitschritt rein elastisch ist, *Trial-Werte* für den neuen Zeitschritt bestimmt:
$$\begin{aligned}
\varepsilon_{vp}^{tr} & = \varepsilon_{vp,n} \\
\sigma^{tr} & = E \left[ \varepsilon - \varepsilon_{vp}^{tr} \right] \\
\Phi^{tr} & = \left| \sigma^{tr} \right| - \sigma_y
\end{aligned}$$

Anschließend wird anhand der Fließfunktion ebendiese Annahme überprüft. Wenn die Bedingung $\Phi^{tr} \leq 0$ erfüllt ist, ist der Zeitschritt tatsächlich rein elastisch, und die Trial-Werte werden als Lösung übernommen:
$$\begin{aligned}
\varepsilon_{vp} & = \varepsilon_{vp}^{tr} \\
\sigma & = \sigma^{tr}
\end{aligned}$$

Andernfalls findet ein viskoplastischer Zeitschritt statt, und die folgende Korrektur wird umgesetzt:
$$\begin{aligned}
\Delta \lambda & = \frac{\Delta t}{\eta + E \Delta t} \Phi^{tr} \\
\varepsilon_{vp} & = \varepsilon_{vp,n} +\Delta \lambda \frac {\sigma^{tr}}{\left|\sigma^{tr}\right|}\\
\sigma & = E \left[ \varepsilon - \varepsilon_{vp} \right|
\end{aligned}$$


# Umsetzung in Python

Um das Verhalten des Modells zu untersuchen, werden wir die oben hergeleiteten Gleichungen in Python implementieren.


## Bestimmung der Spannungen für gegebene Dehnungen

Es soll zunächst eine Funktion erstellt werden, welche die Spannung $\sigma$ und die interne Variable $\varepsilon_{vp}$ zu einer vorgegebenen Dehnung $\varepsilon$ bestimmt. Dazu werden der Funktion die Materialparameter $E$, $\eta$ und $\sigma_y$ sowie die interne Variable aus dem vorherigen Zeitschritt $\varepsilon_{vp,n}$ und die Zeitschrittweite $\Delta t$ übergeben.

---
**Aufgabe**

Vervollständige die Funktion `stress` in der nächsten Zelle.

---

In [ ]:
def stress(eps, epsvp_n, dt, E, sigy, eta):
            
    ...
    
    return sig, epsvp

In [ ]:
from musterloesungen import test_stress
test_stress(stress)

## Beispiel: Zyklische Last, dehnungsgesteuert

Die neue Funktion soll nun verwendet werden, um die Spannungen und internen Variablen während zyklischer Dehnung zu bestimmen. Dabei soll die Dehnung mit einer Dehnrate von $0.1\, \mathrm{s}^{-1}$ auf einen Maximalwert von $0.2$ gesteigert werden. Anschließend soll die Dehnung für $2\,\mathrm{s}$ konstant gehalten werden, bevor die Lastumkehr erfolgt und die Dehnung mit der gleichen Dehnrate wie zuvor bis auf den Wert $-0.2$ gesenkt wird. Nach weiteren $2\,\mathrm{s}$ Haltezeit wird die Dehnung schließlich wieder auf den Wert $0$ gebracht.

---
**Aufgabe**

Vervollständige den Programmcode in der folgenden Zelle, um die gewünschte Dehnung vorzuschreiben.

---

**Hinweis**:
Für diese Aufgabe könnten die numpy-Funktionen [`np.arange`](https://numpy.org/doc/stable/reference/generated/numpy.arange.html) und [`np.heaviside`](https://numpy.org/doc/stable/reference/generated/numpy.heaviside.html) und/oder die Funktion [`scipy.signal.sawtooth`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.sawtooth.html) aus der Bibliothek scipy hilfreich sein...

In [ ]:
# Materialparameter
E = 200.
eta = 50.
sigy = 10.

# Diskrete Zeiten und Dehnungen
tmin = 0.
dt = 0.1

epsmax = 0.2
deps = 0.1 # Rate pro s
Ts = epsmax/deps # Zeit für Rampe
Th = 2. # Haltezeit

T1 = Th
T2 = Th+Ts
T3 = 2*Th+Ts
T4 = 2*Th+3*Ts
T5 = 3*Th+3*Ts
T6 = 3*Th+4*Ts

tmax = 4*Th+4*Ts

t = ...


eps = ...

plt.plot(t, eps)
plt.xlabel(r'Zeit $t$ in $\mathrm{s}$ ')
plt.ylabel(r'Dehnungen $\varepsilon$');


---
**Aufgabe**

Vervollständige den Programmcode in der folgenden Zelle, um zu jeder der vorgeschriebenen Dehnungen die zugehörige Spannung $\sigma$ und interne Variable $\varepsilon_{vp}$ zu bestimmen.

---

**Hinweise**:
- Du solltest die folgende Zelle erst dann bearbeiten, wenn die Dehnungen in der vorherigen Zelle korrekt sind.
- Für den ersten Zeitpunkt $t=0$ gehen wir davon aus, dass die interne Variable den Wert 0 hat.

In [ ]:
# Leere Arrays für Spannung und interne Variable
sig = np.zeros_like(eps)
epsvp = np.zeros_like(eps) # enthält für t=0 bereits die Anfangsbedingung für die interne Variable

# Schleife über Zeitschritte
for i in range(1,len(eps)):
    ...

plt.plot(t, eps, label=r'$\varepsilon$')
plt.plot(t, epsvp, '--', label=r'$\varepsilon_{vp}$')
plt.legend()
plt.xlabel(r'Zeit $t$ in $\mathrm{s}$ ')
plt.ylabel(r'Dehnungen $\varepsilon$')


plt.figure()
plt.plot(t, sig, label=r'$\sigma$')
plt.legend()
plt.xlabel('t');
plt.xlabel(r'Zeit $t$ in $\mathrm{s}$ ')
plt.ylabel(r'Spannungen $\sigma$ in $\mathrm{MPa}$');


# Numerische Lösung des Gleichungssystems

Analog zur Viskoelatizität formulieren wir die Evolutionsgleichung und die Spannungs-Dehnungs-Beziehung im residualen Format.

$$
\mathbf{r} = \begin{bmatrix}
\sigma - E \left[\varepsilon - \varepsilon_{vp}\right] \\
\varepsilon_{vp} - \varepsilon_{vp,n} - \Delta \lambda \dfrac{\sigma}{\left| \sigma \right|}\\
\Delta \lambda \eta - \Delta t \max \left\{0, \Phi\right\} \\
\Phi - \left| \sigma \right| + \sigma_y
\end{bmatrix} = \mathbf 0
\ .$$

Das resultierende Gleichungssystem soll wieder mit der scipy-Funktion `fsolve` gelöst werden.

---
**Aufgabe**

Vervollständige die Funktion `res` zur Bestimmung des Residuums.

---


In [ ]:
# Residuum für gegebene Dehnungen, Spannungen und viskose Verzerrungen
def res(eps, sig, epsvp, dlambda, phi, epsvp_n, dt, E, sigy, eta):
    
    ...
    
    return np.array([r1, r2, r3, r4])


Wie du deine Antwort überprüfst erkläre ich jetzt nicht schon wieder...

In [ ]:
from musterloesungen import test_residuum
test_residuum(res)

## Gleichungslöser

Es gilt nun, die Gleichung $\boldsymbol r = \boldsymbol 0$ nach den Variablen $\sigma$ und $\varepsilon_{vp}$ zu lösen. Wir verwenden wieder die Funktion [`fsolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html) aus dem Paket `scipy`.

---
**Aufgabe**

Vervollständige die Funktion `stress_res` zur Bestimmung der Spannungen und internen Variablen basierend auf der Residuumsfunktion.

---


In [ ]:
# Lösung der Residuumsgleichung nach den Spannungen und viskosen Verzerrungen
def stress_res(eps, epsvp_n, dt, E, sigy, eta, x0 = np.zeros(4)):
    
    ...
    
    return sol[0:2]

In [ ]:
from musterloesungen import test_stress_res
test_stress_res(stress_res)

## Vergleich der numerischen mit der vorherigen Lösung

Im Folgenden werden die beiden Lösungmethoden für den Relaxationsversuch verwendet und die Ergebnisse verglichen. Wenn beide Varianten korrekt implementiert wurden, sollte hier kein Unterschied sichtbar sein.

In [ ]:
# Materialparameter
E = 200.
eta = 50.
sigy = 10.

# Diskrete Zeiten und Dehnungen
tmin = 0.
dt = 0.05

epsmax = 0.2
deps = 0.1 # Rate pro s
Ts = epsmax/deps # Zeit für Rampe
Th = 2. # Haltezeit

T1 = Th
T2 = Th+Ts
T3 = 2*Th+Ts
T4 = 2*Th+3*Ts
T5 = 3*Th+3*Ts
T6 = 3*Th+4*Ts

tmax = 4*Th+4*Ts

t = ...


eps = ...


# Leere Arrays für Spannung und interne Variable
sig = np.zeros_like(eps)
epsvp = np.zeros_like(eps) # enthält für t=0 bereits die Anfangsbedingung für die interne Variable

sig_res = np.zeros_like(eps)
epsvp_res = np.zeros_like(eps) # enthält für t=0 bereits die Anfangsbedingung für die interne Variable


# Schleife über Zeitschritte
for i in range(1,len(eps)):
    ...
    ...
    
plt.figure()
plt.plot(t, eps, 'k', label=r'$\varepsilon$')
plt.plot(t, epsvp, '-', label=r'$\varepsilon_{vp}$')
plt.plot(t, epsvp_res, '--', label=r'$\varepsilon_{vp}$ (residual)')

plt.legend()
plt.xlabel(r'Zeit $t$ in $\mathrm{s}$')
plt.ylabel(r'Dehnungen $\varepsilon$')

plt.figure()
plt.plot(t, sig, label=r'$\sigma$')
plt.plot(t, sig_res, '--', label=r'$\sigma$ (residual)')
plt.legend()
plt.xlabel('t');
plt.xlabel(r'Zeit $t$ in $\mathrm{s}$ ')
plt.ylabel(r'Spannungen $\sigma$ in $\mathrm{MPa}$');


# Inverses Problem

Wie bei der Viskoelastizität kann das residuale Format ausgenutzt werden um ohne große Änderungen die Spannungen vorzuschreiben.

---
**Aufgabe**

Vervollständige die Funktion `strain_res` zur Bestimmung der Dehnung und internen Variablen basierend auf der Residuumsfunktion.

---

In [ ]:
# Lösung der Residuumsgleichung nach den Spannungen und viskosen Verzerrungen
def strain_res(sig, epsvp_n, dt, E, sigy, eta, x0 = np.zeros(4)):
    
    ...
    
    return sol[0:2]

In [ ]:
from musterloesungen import test_strain_res
test_strain_res(strain_res)

Hier wird statt der Dehnung die Spannung vorgeschrieben:

In [ ]:
# Materialparameter
E = 200.
eta = 50.
sigy = 10.

# Diskrete Zeiten und Dehnungen
tmin = 0.
dt = 0.01

sigmax = 20
dsig = 20 # Rate pro s
Ts = sigmax/dsig # Zeit für Rampe
Th = 2. # Haltezeit

T1 = Th
T2 = Th+Ts
T3 = 2*Th+Ts
T4 = 2*Th+3*Ts
T5 = 3*Th+3*Ts
T6 = 3*Th+4*Ts

tmax = 4*Th+4*Ts

t = ...

sig = ...


# Leere Arrays für Spannung und interne Variable
eps = np.zeros_like(sig)
epsvp = np.zeros_like(sig) # enthält für t=0 bereits die Anfangsbedingung für die interne Variable

# Schleife über Zeitschritte
for i in range(1,len(eps)):
    ...

plt.figure()
plt.plot(t, sig, label=r'$\sigma$', linewidth=2.5)
plt.legend()
plt.xlabel(r'Zeit $t$ in $\mathrm{s}$ ')
plt.ylabel(r'Spannungen $\sigma$ in $\mathrm{MPa}$')

plt.figure()
plt.plot(t, eps, label=r'$\varepsilon$', linewidth=2.5)
plt.plot(t, epsvp, '--', label=r'$\varepsilon_v$')
plt.legend()
plt.xlabel(r'Zeit $t$ in $\mathrm{s}$ ')
plt.ylabel(r'Dehnungen $\varepsilon$');


# Weiterführende Aufgaben

Die folgenden Aufgaben können selbstständig bearbeitet werden, um das eigene Verständnis zu verbessern und sich intensiver mit dem Programm auseinander zu setzen

- Erweitere deinen Code, sodass die einzelnen Komponenten der Spannungen geplottet werden. Überlege zunächst, wie dies mit möglichst geringen Änderungen erreicht werden kann.
- Erweitere das letzte Beispiel, so dass im Plot klar wird woraus die Änderungen in der Form der Dehnungskurve resultieren.
- Untersuche den Einfluss verschiedener Parameter analog zur Übung Viskoelastizität
